Erstellen einer FHIR Patient Ressource mit dem Python fhirclient

In [ ]:
!pip install git+git://github.com/smart-on-fhir/client-py.git

In [ ]:
from IPython.display import IFrame
import fhirclient
import fhirclient.models.patient as p

In [ ]:
# Patient Ressource in der FHIR Spezifikation.
IFrame('https://hl7.org/fhir/patient.html#resource', width=1200, height=330)

In [ ]:
# Instanziieren eines Patienten
patient = p.Patient()

In [ ]:
# help() Funktion, um mehr Informationen zu erhalten
help(patient)

In [ ]:
for property in patient.elementProperties():
    print(property)

Name des Patienten / dr Patientin

In [ ]:
IFrame('https://hl7.org/fhir/datatypes.html#HumanName', width=1200, height=330)

In [ ]:
import fhirclient.models.humanname as hn

In [ ]:
name = hn.HumanName()
name.family = 'Bach'
name.given = ['Johann', 'Sebastian']
name.text = 'Johann Sebastian Bach'
name.use = 'official'
patient.name = [name]

Geburtsdatum

In [ ]:
IFrame('https://hl7.org/fhir/datatypes.html#date', width=1200, height=330)

In [ ]:
import fhirclient.models.fhirdate as fd

In [ ]:
dateOfBirth = fd.FHIRDate('1967-05-25')
patient.birthDate = dateOfBirth

In [ ]:
import json
from json import dumps

In [ ]:
print(json.dumps(patient.as_json(), indent=4))

In [ ]:
import requests
from requests import get, post, put

In [ ]:
fhir_test_server = 'https://vonk.fire.ly'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

In [ ]:
response = requests.post(f'{fhir_test_server}/Patient/$validate', headers = headers, data = dumps(patient.as_json()))
data = response.json()


In [ ]:
print(response.status_code)

In [ ]:
print(json.dumps(data, indent=4))

In [ ]:
response = requests.post(f'{fhir_test_server}/Patient', headers = headers, data = dumps(patient.as_json()))
data = response.json()

In [ ]:
print(response.status_code)

In [ ]:
print(json.dumps(data, indent=4))